In [ ]:
# ========================================
# DCGAN con CIFAR-10 - Notebook Completo
# ========================================

# ===== CELDA 1: Imports y Setup =====
import sys
sys.path.append('../src')

from data import load_cifar10_for_gan, get_real_samples
from models import DCGAN
from utils import (plot_generated_images, plot_training_progress, 
                   save_generated_samples, compare_real_vs_fake)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")


# ===== CELDA 2: Configuración =====
CONFIG = {
    'img_size': 32,           # CIFAR-10 es 32x32
    'latent_dim': 100,        # Dimensión del vector de ruido
    'batch_size': 128,        # Tamaño del batch
    'epochs': 100,            # Número de épocas (más para CIFAR-10)
    'sample_interval': 200,   # Cada cuántas iteraciones guardar muestras
    'save_dir': '../results/dcgan_cifar10/'
}

print("\n" + "="*60)
print("CONFIGURACIÓN DEL EXPERIMENTO")
print("="*60)
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


# ===== CELDA 3: Cargar CIFAR-10 =====
print("\n" + "="*60)
print("CARGANDO DATASET CIFAR-10")
print("="*60)

X_train = load_cifar10_for_gan()

print(f"\nDataset shape: {X_train.shape}")
print(f"Número de imágenes: {X_train.shape[0]:,}")
print(f"Memory usage: {X_train.nbytes / (1024**2):.2f} MB")


# ===== CELDA 4: Explorar datos =====
print("\n" + "="*60)
print("EXPLORACIÓN DE DATOS")
print("="*60)

# Nombres de las clases de CIFAR-10
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']

# Mostrar muestras del dataset
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
axes = axes.ravel()

for i in range(32):
    img = X_train[i]
    # Escalar de [-1, 1] a [0, 1] para visualización
    img_display = (img + 1) / 2
    axes[i].imshow(img_display)
    axes[i].axis('off')

plt.suptitle('Muestras del Dataset CIFAR-10', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Rango de valores: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"Shape de una imagen: {X_train[0].shape}")


# ===== CELDA 5: Crear modelo DCGAN =====
print("\n" + "="*60)
print("CREANDO MODELO DCGAN")
print("="*60)

dcgan = DCGAN(
    img_shape=(CONFIG['img_size'], CONFIG['img_size'], 3),
    latent_dim=CONFIG['latent_dim']
)

# Ver arquitectura completa
dcgan.summary()


# ===== CELDA 6: Probar generación inicial (antes de entrenar) =====
print("\n" + "="*60)
print("GENERACIÓN ANTES DE ENTRENAR")
print("="*60)
print("Esto debería verse como ruido aleatorio")

plot_generated_images(
    dcgan.generator, 
    CONFIG['latent_dim'], 
    n_samples=16,
    figsize=(10, 10)
)


# ===== CELDA 7: Función de entrenamiento =====
def train_dcgan(dcgan, X_train, epochs, batch_size, latent_dim, 
                sample_interval, save_dir):
    """
    Entrena el DCGAN
    
    Args:
        dcgan: Instancia del modelo DCGAN
        X_train: Dataset de entrenamiento
        epochs: Número de épocas
        batch_size: Tamaño del batch
        latent_dim: Dimensión del espacio latente
        sample_interval: Cada cuántas iteraciones guardar muestras
        save_dir: Directorio para guardar resultados
    
    Returns:
        d_losses: Lista de losses del discriminador
        g_losses: Lista de losses del generador
        d_accs: Lista de accuracies del discriminador
    """
    import os
    os.makedirs(save_dir, exist_ok=True)
    
    # Labels para entrenamiento
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    
    # Para tracking de métricas
    d_losses = []
    g_losses = []
    d_accs = []
    
    num_batches = X_train.shape[0] // batch_size
    total_iterations = epochs * num_batches
    
    print(f"\n{'='*60}")
    print("INICIANDO ENTRENAMIENTO")
    print(f"{'='*60}")
    print(f"Total épocas: {epochs}")
    print(f"Batches por época: {num_batches}")
    print(f"Total iteraciones: {total_iterations}")
    print(f"{'='*60}\n")
    
    iteration = 0
    
    for epoch in range(epochs):
        print(f"\n{'='*60}")
        print(f"ÉPOCA {epoch + 1}/{epochs}")
        print(f"{'='*60}")
        
        # Shuffle al inicio de cada época
        np.random.shuffle(X_train)
        
        epoch_d_loss = []
        epoch_g_loss = []
        epoch_d_acc = []
        
        for batch_idx in range(num_batches):
            iteration += 1
            
            # ---------------------
            # Entrenar Discriminador
            # ---------------------
            
            # Obtener batch de imágenes reales
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_imgs = X_train[idx]
            
            # Generar imágenes fake
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_imgs = dcgan.generator.predict(noise, verbose=0)
            
            # Entrenar discriminador en reales
            d_loss_real = dcgan.discriminator.train_on_batch(real_imgs, real_labels)
            
            # Entrenar discriminador en fakes
            d_loss_fake = dcgan.discriminator.train_on_batch(fake_imgs, fake_labels)
            
            # Promedio de loss y accuracy
            d_loss = 0.5 * np.add(d_loss_real[0], d_loss_fake[0])
            d_acc = 0.5 * np.add(d_loss_real[1], d_loss_fake[1])
            
            # ---------------------
            # Entrenar Generador
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            
            # Queremos que el discriminador clasifique las fake como reales
            g_loss = dcgan.gan.train_on_batch(noise, real_labels)
            
            # Guardar métricas
            d_losses.append(d_loss)
            g_losses.append(g_loss)
            d_accs.append(d_acc)
            
            epoch_d_loss.append(d_loss)
            epoch_g_loss.append(g_loss)
            epoch_d_acc.append(d_acc)
            
            # Mostrar progreso cada 50 iteraciones
            if iteration % 50 == 0:
                print(f"  [Iter {iteration:05d}/{total_iterations}] "
                      f"D_loss: {d_loss:.4f} | D_acc: {d_acc:.4f} | "
                      f"G_loss: {g_loss:.4f}")
            
            # Guardar muestras periódicamente
            if iteration % sample_interval == 0:
                save_generated_samples(
                    dcgan.generator, 
                    latent_dim, 
                    iteration, 
                    save_dir
                )
                print(f"  ✓ Muestras guardadas: {save_dir}epoch_{iteration:04d}.png")
        
        # Resumen de la época
        avg_d_loss = np.mean(epoch_d_loss)
        avg_g_loss = np.mean(epoch_g_loss)
        avg_d_acc = np.mean(epoch_d_acc)
        
        print(f"\n  Resumen Época {epoch + 1}:")
        print(f"    D_loss promedio: {avg_d_loss:.4f}")
        print(f"    G_loss promedio: {avg_g_loss:.4f}")
        print(f"    D_acc promedio:  {avg_d_acc:.4f}")
    
    print("\n" + "="*60)
    print("✅ ENTRENAMIENTO COMPLETADO")
    print("="*60)
    
    return d_losses, g_losses, d_accs


# ===== CELDA 8: ENTRENAR EL MODELO! =====
print("\n" + "="*60)
print("PREPARANDO ENTRENAMIENTO")
print("="*60)

# Entrenar
d_losses, g_losses, d_accs = train_dcgan(
    dcgan=dcgan,
    X_train=X_train,
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    latent_dim=CONFIG['latent_dim'],
    sample_interval=CONFIG['sample_interval'],
    save_dir=CONFIG['save_dir']
)


# ===== CELDA 9: Visualizar progreso del entrenamiento =====
print("\n" + "="*60)
print("ANÁLISIS DE MÉTRICAS DE ENTRENAMIENTO")
print("="*60)

plot_training_progress(d_losses, g_losses, d_accs)

# Estadísticas
print(f"\nEstadísticas de entrenamiento:")
print(f"  Total iteraciones: {len(d_losses)}")
print(f"\n  Discriminator Loss:")
print(f"    - Primera: {d_losses[0]:.4f}")
print(f"    - Última:  {d_losses[-1]:.4f}")
print(f"    - Mínima:  {min(d_losses):.4f}")
print(f"    - Máxima:  {max(d_losses):.4f}")
print(f"\n  Generator Loss:")
print(f"    - Primera: {g_losses[0]:.4f}")
print(f"    - Última:  {g_losses[-1]:.4f}")
print(f"    - Mínima:  {min(g_losses):.4f}")
print(f"    - Máxima:  {max(g_losses):.4f}")
print(f"\n  Discriminator Accuracy:")
print(f"    - Primera: {d_accs[0]:.4f}")
print(f"    - Última:  {d_accs[-1]:.4f}")
print(f"    - Promedio últimas 100 iter: {np.mean(d_accs[-100:]):.4f}")


# ===== CELDA 10: Generar imágenes finales =====
print("\n" + "="*60)
print("GENERACIÓN FINAL (Después de entrenar)")
print("="*60)

plot_generated_images(
    dcgan.generator, 
    CONFIG['latent_dim'], 
    n_samples=25,
    figsize=(12, 12)
)


# ===== CELDA 11: Comparar real vs generado =====
print("\n" + "="*60)
print("COMPARACIÓN: IMÁGENES REALES VS GENERADAS")
print("="*60)

compare_real_vs_fake(
    X_train, 
    dcgan.generator, 
    CONFIG['latent_dim'], 
    n_samples=10
)


# ===== CELDA 12: Generar imágenes específicas =====
print("\n" + "="*60)
print("GENERANDO MÚLTIPLES VARIACIONES")
print("="*60)

# Generar más imágenes
fig, axes = plt.subplots(8, 8, figsize=(16, 16))
axes = axes.ravel()

noise = np.random.normal(0, 1, (64, CONFIG['latent_dim']))
generated = dcgan.generator.predict(noise, verbose=0)
generated = 0.5 * generated + 0.5
generated = np.clip(generated, 0, 1)

for i in range(64):
    axes[i].imshow(generated[i])
    axes[i].axis('off')

plt.suptitle('64 Imágenes Generadas por DCGAN', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


# ===== CELDA 13: Interpolar en espacio latente =====
print("\n" + "="*60)
print("INTERPOLACIÓN EN ESPACIO LATENTE")
print("="*60)

def interpolate_images(generator, latent_dim, n_steps=10):
    """
    Interpola suavemente entre dos imágenes generadas
    """
    # Dos vectores aleatorios en espacio latente
    z1 = np.random.normal(0, 1, (1, latent_dim))
    z2 = np.random.normal(0, 1, (1, latent_dim))
    
    # Crear interpolación lineal
    alphas = np.linspace(0, 1, n_steps)
    vectors = []
    for alpha in alphas:
        z_interp = z1 * (1 - alpha) + z2 * alpha
        vectors.append(z_interp[0])
    
    vectors = np.array(vectors)
    
    # Generar imágenes
    images = generator.predict(vectors, verbose=0)
    images = 0.5 * images + 0.5
    images = np.clip(images, 0, 1)
    
    # Visualizar
    fig, axes = plt.subplots(1, n_steps, figsize=(20, 2))
    for i in range(n_steps):
        axes[i].imshow(images[i])
        axes[i].axis('off')
        axes[i].set_title(f'{i+1}', fontsize=10)
    
    plt.suptitle('Interpolación Suave en Espacio Latente', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Crear varias interpolaciones
for i in range(3):
    print(f"\nInterpolación {i+1}:")
    interpolate_images(dcgan.generator, CONFIG['latent_dim'], n_steps=10)


# ===== CELDA 14: Guardar modelos =====
print("\n" + "="*60)
print("GUARDANDO MODELOS")
print("="*60)

import os
os.makedirs(CONFIG['save_dir'], exist_ok=True)

# Guardar generador
dcgan.generator.save(f"{CONFIG['save_dir']}generator_final.h5")
print(f"✓ Generador guardado: {CONFIG['save_dir']}generator_final.h5")

# Guardar discriminador
dcgan.discriminator.save(f"{CONFIG['save_dir']}discriminador_final.h5")
print(f"✓ Discriminador guardado: {CONFIG['save_dir']}discriminador_final.h5")

# Guardar métricas
import pickle
metrics = {
    'd_losses': d_losses,
    'g_losses': g_losses,
    'd_accs': d_accs,
    'config': CONFIG
}

with open(f"{CONFIG['save_dir']}training_metrics.pkl", 'wb') as f:
    pickle.dump(metrics, f)

print(f"✓ Métricas guardadas: {CONFIG['save_dir']}training_metrics.pkl")


# ===== CELDA 15: Cargar modelo guardado (para usar después) =====
print("\n" + "="*60)
print("EJEMPLO: CARGAR MODELO GUARDADO")
print("="*60)

# Para cargar después:
"""
from tensorflow.keras.models import load_model

generator_loaded = load_model('../results/dcgan_cifar10/generator_final.h5')

# Generar nuevas imágenes
noise = np.random.normal(0, 1, (16, 100))
new_images = generator_loaded.predict(noise)
new_images = 0.5 * new_images + 0.5

# Mostrar
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
axes = axes.ravel()
for i in range(16):
    axes[i].imshow(new_images[i])
    axes[i].axis('off')
plt.show()
"""

print("Código de ejemplo incluido en comentarios ↑")


# ===== CELDA 16: Resumen final =====
print("\n" + "="*60)
print("RESUMEN FINAL DEL EXPERIMENTO")
print("="*60)

print(f"\n📊 Dataset:")
print(f"  - Nombre: CIFAR-10")
print(f"  - Imágenes: {X_train.shape[0]:,}")
print(f"  - Tamaño: {X_train.shape[1]}x{X_train.shape[2]}")

print(f"\n🏗️  Modelo:")
print(f"  - Arquitectura: DCGAN")
print(f"  - Latent dim: {CONFIG['latent_dim']}")
print(f"  - Parámetros generador: {dcgan.generator.count_params():,}")
print(f"  - Parámetros discriminador: {dcgan.discriminator.count_params():,}")

print(f"\n⚙️  Entrenamiento:")
print(f"  - Épocas: {CONFIG['epochs']}")
print(f"  - Batch size: {CONFIG['batch_size']}")
print(f"  - Total iteraciones: {len(d_losses):,}")

print(f"\n📈 Resultados finales:")
print(f"  - D_loss final: {d_losses[-1]:.4f}")
print(f"  - G_loss final: {g_losses[-1]:.4f}")
print(f"  - D_acc final: {d_accs[-1]:.4f}")
print(f"  - D_acc promedio (últimas 100): {np.mean(d_accs[-100:]):.4f}")

print(f"\n💾 Archivos guardados:")
print(f"  - Generador: {CONFIG['save_dir']}generator_final.h5")
print(f"  - Discriminador: {CONFIG['save_dir']}discriminador_final.h5")
print(f"  - Métricas: {CONFIG['save_dir']}training_metrics.pkl")
print(f"  - Muestras: {CONFIG['save_dir']}epoch_*.png")

print("\n" + "="*60)
print("✅ NOTEBOOK COMPLETADO EXITOSAMENTE")
print("="*60)